In [3]:
from datetime import datetime
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
from pandasql import sqldf
import re

In [30]:
# let's first make a function that will make the process of building a url easy.
def make_url(base_url , comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url, r)
        
    return url

# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    return(np.unique(x)) 

In [145]:
start = time.time()

#define our base url 
base_url = r"https://www.sec.gov/Archives/edgar/data"
cik_num = '/1318605/'


#lets create a filing url 
filings_url = base_url +cik_num + '/index.json'
content = requests.get(filings_url)
decoded_content = content.json()

filings = decoded_content['directory']['item']

summary_filing_list = []

#iterate through each filing for GS
for filing in filings: 
    
    #define each filing number 
    filing_num = filing['name']
    
    filing_url = base_url + cik_num + filing_num + '/index.json'
    print("Filing URL")
    print(filing_url)
    
    try: 
        content = requests.get(filing_url)
        sleep(0.2)
        document_content = content.json()
        
        doc_directory = pd.DataFrame(document_content['directory']['item'])
        
        deduped_doc_directory = doc_directory.drop_duplicates(subset = ['name'])
        #print(deduped_doc_directory )
        
        #iterate through each document in each filing 
        for index,document in deduped_doc_directory.iterrows():
            #print(document['name'])
            if document['type'] != 'image2.gif' and '.txt' in document['name']:
                doc_name = document['name']
                document_url = base_url + cik_num + filing_num + '/index.json'
#                 print(document)
                try: 
                    sleep(0.2)
                    doc_content = requests.get(document_url).json()
                    doc_content_df = pd.DataFrame(doc_content['directory']['item'])
                    #print(doc_content_df)
                    
                    deduped_doc_content = doc_content_df.drop_duplicates(subset = ['name'])
                    #print(deduped_doc_content)
                    #doc_content = content.json()
                    #print(doc_content)
                    #print(document_url)
                    
                    for index, file in deduped_doc_content.iterrows():
                        if file['name'] == 'FilingSummary.xml':
                            #print(file['name'])
                            xml_summary = "https://www.sec.gov" + doc_content['directory']['name'] +'/' + file['name']
                            #print(xml_summary)
                            summary_filing_list.append(xml_summary)
                except Exception:
                    print("failed to get filing summaries")
                    pass
                    
    except Exception: 
        print("failed to get json:" + filing_url)

#dedupe unique filing summaries and store in dataframe 
summary_filing_df = pd.DataFrame(summary_filing_list, columns = ['xml_summary'])
summary_filing_df = summary_filing_df.drop_duplicates() 
print(summary_filing_df)

end = time.time()
print(end - start)
print("finished compiling summary urls")

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000179056520000008/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459020032196/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000158451820000008/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459020031568/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459020031314/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000181123020000003/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459020030017/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000121465920005791/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000121465920005699/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459020029536/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000179056520000007/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000177134019000009/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000179056519000001/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459019036274/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459019035971/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/999999999719007129/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/999999999719007092/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000177136419000013/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000000000019013672/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/999999999719006839/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000158451819000007/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000177136419000012/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000177136419000001/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000158451819000005/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312519075818/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312519075806/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312519075803/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312519075801/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312519075758/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515819000003/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459019007669/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312519070100/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459019007080/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515818000003/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473218000007/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473218000006/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473218000005/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515818000002/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515818000001/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473018000005/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000165064918000001/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473218000004/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459018015129/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000137773918000032/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000135281617000005/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000135281617000004/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000135281617000003/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000135281617000002/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000135281617000001/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459017013548/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459017013276/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473117000008/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459017013176/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459017013123/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000156459017012932/index.json
Filing URL
https://ww

...

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473114000004/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000128825714000027/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312514297582/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312514290187/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312514289598/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312514228961/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515814000016/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515814000014/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312514222060/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515814000011/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515814000009/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513222426/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513222376/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513222292/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513212354/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513207910/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149472713000008/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473113000009/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/999999999713008705/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513158922/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312513158904/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473013000002/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515812000002/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149480612000005/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149515812000001/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000123404612000003/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149472712000009/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149472712000008/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312512081990/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149480612000004/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149472712000006/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473212000004/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000123404612000002/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312511126139/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473211000017/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149472711000010/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473211000015/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/999999999711008519/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312511092514/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312511092509/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000031506611003331/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/999999999711007341/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149472711000008/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000149473211000013/index.json
Filing URL
https://ww

Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312510135111/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000095013010002906/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312510129881/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312510129878/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000000000010027152/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312510099603/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000000000010019954/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312510068933/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000000000010010920/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000119312510017054/index.json
Filing URL
https://www.sec.gov/Archives/edgar/data/1318605/000131860509000006/index.json
Filing URL
https://ww

In [ ]:
#ingest summary df and pull reports, write to csv 
def pull_reports(df):
        #iterate through df and get financial data reports in a master list of dictionaries
    print("Starting to pull reports")
    start = time.time()

    master_reports = []

    stmt_list = []

    category_list = []
    for index, row in df.iterrows():
        try: 
            xml = row['xml_summary']
            base_url = row['xml_summary'].replace('FilingSummary.xml', '')
            #print(xml)
            content = requests.get(xml).content
            sleep(0.25)
            soup = BeautifulSoup(content, 'lxml')
            # find the 'myreports' tag because this contains all the individual reports submitted.
            reports = soup.find('myreports') 
            # print(reports) 
            # loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
            for report in reports.find_all('report')[:-1]:

                # let's create a dictionary to store all the different parts we need.
                report_dict = {}
                report_dict['name_short'] = report.shortname.text.lower()
                report_dict['name_long'] = report.longname.text

                # position and category isnt always available
                try: 
                    report_dict['position'] = report.position.text
                    report_dict['category'] = report.menucategory.text
                except Exception as ex:
                    report_dict['position'] = ''
                    report_dict['category'] = ''

                report_dict['url'] = base_url + report.htmlfilename.text

                # only add statements accounting for missing categories
                if (report_dict['category'] == "Statements" or report_dict['category'] =='') and "consolidated" in report_dict['name_short'] and "unaudited" not in report_dict['name_short']: 

                    # append the dictionary to the master list.
                    master_reports.append(report_dict)

#                     stmt_list.append(report_dict['name_short'])
#                     category_list.append(report_dict['category'])
                    #print(report_dict)
                else: 
                    pass
        except Exception as ex:
            print("failed")

    #         # print the info to the user.
    #         print('-'*100)
    #         print(base_url + report.htmlfilename.text)
    #         print(report.longname.text)
    #         print(report.shortname.text)
    #         print(report.menucategory.text)
    #         print(report.position.text)

    #End Code
    end = time.time()
    print(end - start)
    print("Finished Pulling Reports")
    
    print("Writing reports to csv")
    #write reports to csv
    pd.DataFrame(master_reports).to_csv("example_master_reports.csv")
    
    #pull from csv
    master_reports_extract = pd.read_csv("example_master_reports.csv")
    print(master_reports_extract)
    return(master_reports_extract)
    

In [92]:
report_df = pull_reports(summary_filing_df)

Starting to pull reports
20.694371700286865
Finished Pulling Reports
Writing reports to csv
     Unnamed: 0                                         name_short  \
0             0                        consolidated balance sheets   
1             1        consolidated balance sheets (parenthetical)   
2             2              consolidated statements of operations   
3             3      consolidated statements of comprehensive loss   
4             4  consolidated statements of redeemable noncontr...   
..          ...                                                ...   
125         125    condensed consolidated statements of cash flows   
126         126              condensed consolidated balance sheets   
127         127  condensed consolidated balance sheets (parenth...   
128         128    condensed consolidated statements of operations   
129         129    condensed consolidated statements of cash flows   

                                             name_long  position   

In [93]:
pd.DataFrame(report_df).to_csv("example_master_reports.csv")

#pull from csv
report_extract = pd.read_csv("example_master_reports.csv")

In [94]:
print(report_extract)

     Unnamed: 0  Unnamed: 0.1  \
0             0             0   
1             1             1   
2             2             2   
3             3             3   
4             4             4   
..          ...           ...   
125         125           125   
126         126           126   
127         127           127   
128         128           128   
129         129           129   

                                            name_short  \
0                          consolidated balance sheets   
1          consolidated balance sheets (parenthetical)   
2                consolidated statements of operations   
3        consolidated statements of comprehensive loss   
4    consolidated statements of redeemable noncontr...   
..                                                 ...   
125    condensed consolidated statements of cash flows   
126              condensed consolidated balance sheets   
127  condensed consolidated balance sheets (parenth...   
128    condensed consol

In [ ]:
def parse_reports(df):
    